<a href="https://colab.research.google.com/github/DariusBotusanu/RoFinBert/blob/main/fine_tuning_colab_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install transformers datasets evaluate
! pip install accelerate -U

In [2]:
###IMPORTS###
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
import evaluate

In [3]:
dataset = load_dataset("dariusbotusanu/ro_financial_phrasebank_v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3168
    })
    test_all_agree: Dataset({
        features: ['text', 'label'],
        num_rows: 680
    })
    test_75_agree: Dataset({
        features: ['text', 'label'],
        num_rows: 680
    })
})

In [5]:
tokenizer = AutoTokenizer.from_pretrained("racai/distilbert-base-romanian-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/680 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [6]:
model = AutoModelForSequenceClassification.from_pretrained("racai/distilbert-base-romanian-cased", num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at racai/distilbert-base-romanian-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)



training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test_all_agree'],
    compute_metrics=compute_metrics,
)

In [9]:
trainer.train()

ValueError: expected sequence of length 12 at dim 1 (got 41)

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3168
    })
    test_all_agree: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 680
    })
    test_75_agree: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 680
    })
})